# Travelling Salesman Problem with Tensor Networks


Vamos a implementar el TSP con tensor networks.

In [3]:
import torch
import tensorkrowch as tk
import numpy as np

from IPython.display import HTML, display

---
# Función global

In [4]:
def TSP_solver(distances:np.array, tau:float=1, verbose:bool=True, n_layers:int=1):
    """
    Solves the TSP using tensor networks.
    Parameters:
        distances (np.array): Array of distances between cities.
        tau (float): Imaginary time parameter.
    Returns:
        list: List of cities in the optimal path.
    """
    
    # Numero de ciudades
    N_cities = distances.shape[0]
    remaining_cities = np.arange(N_cities - 1)

    if verbose:
        out = display(progress(0, N_cities), display_id=True)
        var_counter = 0

    #Vector solución
    solution = np.zeros(N_cities, dtype=int)
    solution[0] = N_cities - 1

    if verbose:
        var_counter += 1
        out.update(progress(var_counter, N_cities))
    


    for t in range(1, N_cities-1):
        new_distances = np.ones((N_cities - t + 2, N_cities - t + 2), dtype=float)*np.inf
        for i, city_origen in enumerate(remaining_cities):
            for j, city_destino in enumerate(remaining_cities):
                new_distances[i, j] = distances[city_origen, city_destino]
            
            new_distances[i, -1] = distances[city_origen, -1]
        
        for j, city_destino in enumerate(remaining_cities):
            new_distances[-2, j] = distances[solution[t-1], city_destino]
        
        if t != 1: new_distances[-2, -1] = distances[solution[t-1], -1]

        # Obtenemos la solución parcial con el mapeado impuesto al grafo recortado
        vector_solution = TSP_solver_from_to(new_distances, tau, n_layers)

        if max(vector_solution) < 1e-120: break
        
        # Obtener la solución
        partial_solution = np.argmax(vector_solution)

        # Mapeamos la solución parcial a la solución global
        solution[t] = remaining_cities[partial_solution]

        # Actualizamos el vector de ciudades restantes
        remaining_cities = np.delete(remaining_cities, partial_solution)
        
        if verbose:
            var_counter += 1
            out.update(progress(var_counter, N_cities))

    # Último paso
    solution[-1] = remaining_cities[0]
    if verbose:
        var_counter += 1
        out.update(progress(var_counter, N_cities))

    return solution



Función TSP desde uno inicial a uno final.

In [5]:
def TSP_solver_from_to(distances:np.array, tau:float, n_layers:int):

    # Crear la tensor network
    tn = tk.TensorNetwork(name='TSP')

    # Creamos los tensores
    tensor_network = create_tensors(tn, distances, tau, n_layers)

    # Contracción de la tensor network
    vector_solution = contract_tensors(tensor_network)

    return vector_solution

---
# Función de Tensor Network


## Función creadora de tensores

In [6]:
def create_tensors(tn:tk.TensorNetwork, distances:np.array, tau:float, n_layers:int):
    # Numero de nodos en el subproblema
    n_nodes = len(distances)-2
    # Capa de inicialización
    s_layer = superposition_layer(tn, n_nodes)

    # Capa de evolución en tiempo imaginario
    e_layer = evolution_layer(tn, n_nodes, distances, tau)

    # Capa de restricción
    r_layer = []
    append = r_layer.append
    if n_layers >= n_nodes-1:
        for node in range(n_nodes-1):
            append( restriction_layer(tn, n_nodes, node))
    else:
        condition_index = np.arange(n_nodes-1)
        for _ in range(n_layers):
            node = np.random.choice(condition_index)
            append( restriction_layer(tn, n_nodes, node))
            condition_index = np.delete(condition_index, np.where(condition_index == node))

    #Capa de traceado
    t_layer = trace_layer(tn, n_nodes)

    for node in range(n_nodes):
        s_layer[node]['right'] ^ e_layer[node]['left']
        if n_layers != 0:
            e_layer[node]['right'] ^ r_layer[0][node]['left']
            for depth in range(min(n_nodes-2,n_layers-1)):
                r_layer[depth][node]['right'] ^ r_layer[depth+1][node]['left']
            r_layer[-1][node]['right'] ^ t_layer[node]['left']
        else:
            e_layer[node]['right'] ^ t_layer[node]['left']

    if n_layers == 0:
        return [s_layer, e_layer, t_layer]
    
    return [s_layer, e_layer] + r_layer + [t_layer]

### Capa de superposición

In [7]:
def superposition_layer(tn:tk.TensorNetwork, n_nodes:int):

    uniform_node = tk.Node(tensor=torch.ones(n_nodes), name='uniform', axes_names=['right'], network=tn, virtual=True)

    layer = []
    append = layer.append

    for i in range(n_nodes):
        append(   tk.Node(shape=(n_nodes,), name=f'initial_({i},{0})', axes_names=['right'], network=tn)     )
        layer[i].set_tensor_from(uniform_node)

    #stack_nodes = tk.stack(layer)

    return layer

### Capa de evolución

In [8]:
def evolution_layer(tn: tk.TensorNetwork, n_nodes:int, distances:np.array, tau:float):
    # Inicializamos la lista que contendrá los nodos de la capa de evolución
    layer = []
    append = layer.append

    # Creamos el tensor para el nodo inicial
    initial_tensor = torch.zeros(size=(n_nodes, n_nodes, n_nodes))
    for current in range(n_nodes):
        # Calculamos la exponencial negativa de la distancia multiplicada por tau
        initial_tensor[current, current, current] = np.exp(-tau*distances[n_nodes,current])
    # Creamos el nodo inicial y lo añadimos a la capa
    initial_node = tk.Node(tensor=initial_tensor, name=f'evolution_({0},{1})', network=tn, axes_names=['left', 'right', 'down'])
    append(initial_node)

    # Creamos el tensor para los nodos intermedios
    intermediate_tensor = torch.zeros(size=(n_nodes, n_nodes, n_nodes, n_nodes))
    for previous in range(n_nodes):
        for current in range(n_nodes):
            # Calculamos la exponencial negativa de la distancia multiplicada por tau
            intermediate_tensor[current, current, previous, current] = np.exp(-tau*distances[previous,current])
    # Creamos un nodo virtual con el tensor intermedio
    intermediate_node = tk.Node(tensor=intermediate_tensor, name=f'evolution_(uniform)', network=tn, virtual=True)

    # Creamos los nodos intermedios y los añadimos a la capa
    for node in range(1, n_nodes-1):
        append(  tk.Node(shape=(n_nodes, n_nodes, n_nodes, n_nodes), name=f'evolution_({node},{1})', network=tn, axes_names=['left', 'right', 'up', 'down'])   )
        layer[node].set_tensor_from(intermediate_node)

        layer[node]['up'] ^ layer[node-1]['down']

    # Creamos el tensor para el nodo final
    final_tensor = torch.zeros(size=(n_nodes, n_nodes, n_nodes))
    for previous in range(n_nodes):
        for current in range(n_nodes):
            # Calculamos la exponencial negativa de la suma de distancias multiplicada por tau
            final_tensor[current, current, previous] = np.exp(-tau*(distances[previous,current]+distances[current, n_nodes+1]))
    # Creamos el nodo final y lo añadimos a la capa
    final_node = tk.Node(tensor=final_tensor, name=f'evolution_({n_nodes-1},{1})', network=tn, axes_names=['left', 'right', 'up'])
    append(final_node)
    
    layer[-1]['up'] ^ layer[-2]['down']

    # Apilamos todos los nodos de la capa
    #stack_nodes = tk.stack(layer)

    return layer


### Capas de restricción

In [9]:
def restriction_layer(tn:tk.TensorNetwork, n_nodes:int, node:int):
    # Inicializamos la lista que contendrá los nodos de la capa de restricción
    layer = []
    append = layer.append

    # Creamos el tensor para el nodo inicial
    initial_tensor = torch.zeros(size=(n_nodes, n_nodes, 2))
    for current in range(n_nodes):
        if current == node: initial_tensor[node, node, 1] = 1
        else:               initial_tensor[current, current, 0] = 1
    
    # Creamos el nodo inicial y lo añadimos a la capa
    initial_node = tk.Node(tensor=initial_tensor, name=f'restr_({node},{0})', network=tn, axes_names=['left', 'right', 'down'])
    append(initial_node)

    # Creamos el tensor para los nodos intermedios
    intermediate_tensor = torch.zeros(size=(n_nodes, n_nodes, 2, 2))
    for current in range(n_nodes):
        if current == node:
            intermediate_tensor[current, current, 0, 1] = 1
        else:
            intermediate_tensor[current, current, 0, 0] = 1
            intermediate_tensor[current, current, 1, 1] = 1
    
    # Creamos un nodo virtual con el tensor intermedio
    intermediate_node = tk.Node(tensor=intermediate_tensor, name=f'restr_(uniform)', network=tn, virtual=True)

    # Creamos los nodos intermedios y los añadimos a la capa
    for i_node in range(1, n_nodes-1):
        append(  tk.Node(shape=(n_nodes, n_nodes, 2, 2), name=f'restr_({node},{i_node})', network=tn, axes_names=['left', 'right', 'up', 'down'])   )
        layer[i_node].set_tensor_from(intermediate_node)

        layer[i_node]['up'] ^ layer[i_node-1]['down']

    # Creamos el tensor para el nodo final
    final_tensor = torch.zeros(size=(n_nodes, n_nodes, 2))
    for current in range(n_nodes):
        if current == node: final_tensor[current, current, 0] = 1
        else:               final_tensor[current, current, 1] = 1

    # Creamos el nodo final y lo añadimos a la capa
    final_node = tk.Node(tensor=final_tensor, name=f'restr_({node},{n_nodes-1})', network=tn, axes_names=['left', 'right', 'up'])
    append(final_node)
    
    layer[-1]['up'] ^ layer[-2]['down']

    # Apilamos todos los nodos de la capa
    #stack_nodes = tk.stack(layer)

    return layer

### Capa de traceado

In [10]:
def trace_layer(tn:tk.TensorNetwork, n_nodes:int):

    uniform_node = tk.Node(tensor=torch.ones(n_nodes), name='uniform', axes_names=['left'], network=tn, virtual=True)

    layer = []
    append = layer.append

    for i in range(n_nodes):
        append(   tk.Node(shape=(n_nodes,), name=f'trace_({i})', axes_names=['left'], network=tn)     )
        layer[i].set_tensor_from(uniform_node)

    #stack_nodes = tk.stack(layer)

    return layer

## Función de contracción

In [11]:
def contract_tensors(tensor_network:list[list[tk.Node]]):
    # Contraemos la capa de superposición con la capa de evolución´
    n_qudits = len(tensor_network[0])
    depth = len(tensor_network)
    # Extremos
    for qudit in range(n_qudits):
        tensor_network[1][qudit] = tk.contract_between_(tensor_network[0][qudit], tensor_network[1][qudit])
        tensor_network[1][qudit].name = f'Initial_evolution_({qudit})'

        if qudit != 0:
            tensor_network[-2][qudit] = tk.contract_between_(tensor_network[-1][qudit], tensor_network[-2][qudit])
            tensor_network[-2][qudit].name = f'trace_restr_({qudit})'
    
    # Contraccion de ultimo qudit
    result = tensor_network[-2][-1]
    
    for layer in range(depth-3, 0, -1):
        result = tk.contract_between_(result, tensor_network[layer][-1])

    # Contraccion de los demas qudits
    for qudit in range(n_qudits-2, -1, -1):
        for layer in range(1, depth-1):
            result = tk.contract_between_(result, tensor_network[layer][qudit])

    return result.tensor.flatten()


---
# Funciones del problema

In [12]:
def generate_problem(n_cities:int, n_conexions:int, distance_range:float):
    distances = torch.ones(size=(n_cities, n_cities), dtype=int)*np.inf

    for previous in range(n_cities):
        possible_destinies = np.delete(np.arange(n_cities, dtype=int), previous)

        for destiny in range(n_conexions):
            current = np.random.choice(possible_destinies)
            distances[previous, current] = np.random.randint(1,distance_range)
            possible_destinies = np.delete(possible_destinies, np.where(possible_destinies == current))

    return distances

def cost_solution(solution:np.array, distances:np.array):
    cost = 0
    for i in range(len(solution)-1):
        cost += distances[solution[i], solution[i+1]]

    cost += distances[solution[-1], solution[0]]

    return cost

def progress(value, max=100):
    return HTML("""
                <progress
                value='{value}'
                max='{max}'
                style='width: 50%'
                >
                {value}
                </progress>
                <p> Progreso: {porc} %</p>
                """.format(value=value, max=max, porc=np.round(value/max*100,2)))

---
# Prueba

In [13]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model(distances, n_cities):
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [[ int(_.numpy().flatten()[0]) if _ < np.inf else 30000 for _ in origin_list] for origin_list in distances]
    data["num_vehicles"] = 1
    data["depot"] = n_cities - 1
    return data


def print_solution(manager, routing, solution, distances):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()} miles")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    route = [index]
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route.append(index)
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f" {manager.IndexToNode(index)}\n"
    print(plan_output)
    plan_output += f"Route distance: {route_distance}miles\n"
    print(route[:-1])
    print(chr(27)+"[1;33m"+'Coste: '+str(cost_solution(route[:-1], distances)))





def solucion_Google(distances, n_cities):
    """Simple Travelling Salesperson Problem (TSP) between cities."""
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(distances, n_cities)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution, distances)


In [16]:
n_cities = 10
n_conexions = 5
distance_range = 10
tau = 1e0
n_layers = 11

distances = generate_problem(n_cities, n_conexions, distance_range)

solution = TSP_solver(distances, tau=tau, n_layers=n_layers)

print('Solucion: ', solution)
print(chr(27)+"[1;33m"+'Coste de solucion: '+str(cost_solution(solution, distances)))

print('='*100)
solucion_Google(distances, n_cities)


Solucion:  [9 7 6 3 0 8 4 2 1 5]
Coste de solucion: tensor(27.)
Objective: 32 miles
Route for vehicle 0:
 9 -> 8 -> 2 -> 4 -> 5 -> 0 -> 3 -> 7 -> 6 -> 1 -> 9

[9, 8, 2, 4, 5, 0, 3, 7, 6, 1]
Coste: tensor(32.)


In [17]:
activate = False

# Advertencia: Este programa demuestra que P ≠ NP, no que P = NP
if activate:
    import itertools

    def tsp_fuerza_bruta(distancias):
        n = len(distancias)
        ciudades = range(n)
        mejor_ruta = None
        mejor_distancia = float('inf')
        
        for permutacion in itertools.permutations(ciudades[1:]):
            ruta = (0,) + permutacion + (0,)
            distancia_total = sum(distancias[ruta[i]][ruta[i+1]] for i in range(n))
            
            if distancia_total < mejor_distancia:
                mejor_distancia = distancia_total
                mejor_ruta = ruta
        
        return mejor_ruta[:-1], mejor_distancia

    # Usar la función
    solucion_optima, distancia_optima = tsp_fuerza_bruta(data["distance_matrix"])

    print("Solución óptima encontrada por fuerza bruta:")
    print("Ruta:", solucion_optima)
    print("Distancia total:", distancia_optima)

    # Nota: Este enfoque tiene una complejidad de O(n!), lo que demuestra que P ≠ NP
    # para el problema del TSP, ya que no se conoce un algoritmo polinomial para resolverlo.

    tsp_fuerza_bruta(distances)

In [ ]:
tk.Node